In [38]:
import pandas as pd
from konlpy.tag import Mecab
    # 동사/명사등 형태소 분석을 위해 Mecab을 import한다. 
from gensim import corpora, models
    # gensim라이브러리에서 말뭉치를 다루는 corpora클래스와 LDA모델링을 수행하는 models를 import한다.
from nltk.tokenize import sent_tokenize
    # 뉴스기사를 문장 단위로 분리하기 위해 sent_tokenize함수를 불러온다.

df = pd.read_csv('.\clean_IT_news.csv').head(15) #데이터프레임을 만들어서 15개의 데이터만 로딩한다.
                                                #데이터프레임에서 로딩 개수는 마음대로 조정할 수 있다.

with open('230406_송세영님_stop_word.txt', 'r', encoding='utf-8') as f:
    korean_stop_words = set(line.strip() for line in f)
    # 송세영님이 작성한 불용어 파일을 불러와서 korean_stop_words에 할당한다.

def preprocess(text):
    mecab = Mecab(dicpath=r"C:/mecab/mecab-ko-dic")
    tokens = [word for word in mecab.pos(text) if word[1][0] in ['N', 'V']]
    # Mecab을 이용해서 문장에서 명사와 동사 품사만 추출한다.
    tokens = [word for word, pos in tokens if word not in korean_stop_words]
    # 추출한 단어중에서 송세영님이 작성한 불용어를 제외한 단어 리스트를 반환한다.
    return tokens

def remove_sources(text, sources=['출처=IT동아', '사진=트위터 @DylanXitton']):
    # 송세영님이 작성한 불용어 파일에 없는 단어들중에 필요없는 문구를 제거해 주기 위한 함수이다.
    for source in sources:
        text = text.replace(source, '')
    return text

def lda_summarize(text, num_topics=3, num_words=10, num_sentences=3):
        # 요약문을 생성하는 함수 정의, text는 요약 대상 텍스트, num_topics는 토픽 수, 
        # num_words는 토픽당 키워드 수, num_sentences는 요약문에 포함될 문장 수입니다.
    text = remove_sources(text)
        # remove_sources함수를 사용하여 sources에 지정된 문구를 제거하고 다시 text변수에 할당합니다.
    sentences = sent_tokenize(text)
        # text를 nltk.tokenize에서 import한 sent_tokenize()함수를 이용해서 문장으로 나누어 줍니다.
    tokenized_sentences = [preprocess(sentence) for sentence in sentences]
        # 문장을 전처리하고 토큰화합니다.
    dictionary = corpora.Dictionary(tokenized_sentences)
        # 모든 단어를 고유한 정수로 부여한 ID에 매핑합니다.
    corpus = [dictionary.doc2bow(tokenized_sentence) for tokenized_sentence in tokenized_sentences]
        # 각 문서의 단어 출현 빈도를 매핑된 단어 ID에 대한 (단어 ID, 단어 빈도) 쌍의 [리스트]로 변환합니다.
    lda_model = models.ldamodel.LdaModel(corpus, num_topics=num_topics, id2word=dictionary, passes=20)
        # gensim라이브러리에서 임포트한 models를 사용해서 LDA 모델을 생성하고 학습시킵니다
    topic_keywords = lda_model.show_topics(num_topics=num_topics, num_words=num_words, formatted=False)
        # 각 토픽의 단어 분포와 해당 단어들의 점수를 반환합니다.
    topic_sentences = []
        # 요약문에 포함될 토픽과 관련된 문장을 저장하기 위한 리스트를 생성합니다.
    for topic, keywords in topic_keywords:
            # 모든 토픽과 관련된 단어들을 for문으로 순서대로 돌립니다.
        topic_words = [word[0] for word in keywords]
            # 토픽의 키워드를 리스트로 생성합니다.
        for sentence in sentences:
            # 문장을 for문으로 돌립니다.
            tokenized_sentence = preprocess(sentence)
                # 현재 문장을 전처리해서 토큰화합니다.
            if any(word in tokenized_sentence for word in topic_words):
                # 현재 문장에 토픽의 키워드중에서 하나가 있으면,,,, 
                topic_sentences.append(sentence)
                # 현재 문장을 topic_sentences 리스트에 추가해 줍니다.
    ranked_sentences = sorted([(sentence, lda_model[corpus[sentences.index(sentence)]][0][1]) for sentence in topic_sentences], key=lambda x: x[1], reverse=True)
                # 모든 topic_sentences에 대해 해당 문장이 속한 토픽의 점수를 계산하고, 점수가 높은 순서로 정렬합니다.
    summary = [sentence for sentence, score in ranked_sentences][:num_sentences]
                # 점수가 가장 높은 num_sentences개의 문장을 선택할 수 있습니다.
    return summary

for index, row in df.iterrows():
        # 데이터프레임에서 각각의 행과 인덱스를 반복적으로 돌립니다.
    content = row['content']
        # 각 행에서 해당 뉴스기사의 본문인 content를 추출해서 lda_summarize()함수를 호출합니다.
    summary = lda_summarize(content, num_topics=3, num_words=10, num_sentences=3)
        # 매개변수 content는 기사내용, num_topics는 생성할 토픽의 수, num_words는 단어의 개수, nun_sentences는 문장의 수입니다.
    print(f"{index+1}. 기사 제목: {row['title']}")
    print(f"◆기사요약문◆  \n{' / '.join(summary)}")
    print('-'*135)


1. 기사 제목: 벤츠 “미래 비전 담은 12종 신차 내놓을 것”…첫 포문 ‘더 뉴 EQS SUV’ 출시
◆기사요약문◆  
이날 출시행사에서 차량을 소개한 요하네스 슌 벤츠코리아 제품, 마케팅 및 디지털 비즈니스 총괄은 "자사 전기차 최초로 더 뉴 EQS SUV에 적용한 오프로드 주행모드를 통해 비포장도로나 경사면을 달릴 때도 최적의 드라이빙 시스템을 제공하겠다"며 "오프로드 모드 시 주행 속도에 따라 전고가 최대 25㎜ 높아지며, 회전할 가능성이 높은 바퀴의 미끄러짐을 최소화하거나 추진력을 위해 충분히 미끄러짐을 허용하는 방식으로 차량을 안전하고 쉽게 제어하도록 도울 것"이라고 말했다. / 이날 출시행사에서 차량을 소개한 요하네스 슌 벤츠코리아 제품, 마케팅 및 디지털 비즈니스 총괄은 "자사 전기차 최초로 더 뉴 EQS SUV에 적용한 오프로드 주행모드를 통해 비포장도로나 경사면을 달릴 때도 최적의 드라이빙 시스템을 제공하겠다"며 "오프로드 모드 시 주행 속도에 따라 전고가 최대 25㎜ 높아지며, 회전할 가능성이 높은 바퀴의 미끄러짐을 최소화하거나 추진력을 위해 충분히 미끄러짐을 허용하는 방식으로 차량을 안전하고 쉽게 제어하도록 도울 것"이라고 말했다. / 이날 출시행사에서 차량을 소개한 요하네스 슌 벤츠코리아 제품, 마케팅 및 디지털 비즈니스 총괄은 "자사 전기차 최초로 더 뉴 EQS SUV에 적용한 오프로드 주행모드를 통해 비포장도로나 경사면을 달릴 때도 최적의 드라이빙 시스템을 제공하겠다"며 "오프로드 모드 시 주행 속도에 따라 전고가 최대 25㎜ 높아지며, 회전할 가능성이 높은 바퀴의 미끄러짐을 최소화하거나 추진력을 위해 충분히 미끄러짐을 허용하는 방식으로 차량을 안전하고 쉽게 제어하도록 도울 것"이라고 말했다.
---------------------------------------------------------------------------------------------------------------------------------------